In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
import base64
from io import BytesIO
from PIL import Image
import requests
import os
import shutil
import random

In [80]:
DRIVER_PATH = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
# base_url='https://psl.org.pk/dictionary/category/'


C:\Users\dell\AppData\Local\Temp\ipykernel_5748\3378463292.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [87]:
#excluding unwanted terms from classes
file1 = open("dict.txt", "r")
dictionaryTerms=(file1.read()).split('\n')
file1.close()

file2 = open("excludedTerms.txt", "r")
excludedTerms=(file2.read()).split('\n')
file2.close()

classes=[x for x in dictionaryTerms if x not in excludedTerms]
classes=classes[133:]
# classes[459]
#classes=['Eyeglasses','Shoe','Socks','Eggs','Wall Clock']
# print(classes[131])


In [17]:
def get_links(item:str)->list:
    item=item.replace(' ','+')
    base_url='https://www.google.com/search?rlz=1C1RLNS_enPK1029PK1029&sxsrf=AJOqlzVVNNSw1Vg1KScun73k7uuQyLzbtw:1677766795088&q='+item+'&tbm=isch&sa=X&ved=2ahUKEwiHydiuuL39AhVohf0HHW5ZBccQ0pQJegQIChAB&biw=1366&bih=625&dpr=1'
    links=[]
    driver.get(base_url)
    driver.execute_script("window.scrollBy(0,1500)")

    elements=(driver.find_elements(By.CLASS_NAME, 'rg_i'))
    for e in elements:
        links.append(e.get_attribute('src'))
    return links

In [18]:
def separate_links(links):
    downloadable=[]
    base_64_images=[]
    for i,link in enumerate(links):
            if link is not None :
                if 'base64' in link:
                    base_64_images.append(i)
                else:
                    downloadable.append(i)
    return downloadable,base_64_images

In [19]:

def download_img(downloadable,links,path):
    for index in downloadable:
        response = requests.get(links[index])
        with open(path+'/img'+str(index)+'.jpg', 'wb') as f:
            f.write(response.content)

In [20]:

def save_base64(base_64_images,links,path):
    for index in base_64_images:
        base64_image =links[index]
        image_data = base64_image.split(',')[1]
        decoded_image_data = base64.b64decode(image_data)
        image_buffer = BytesIO(decoded_image_data)
        image = Image.open(image_buffer)
        image = image.convert('RGB')
        image.save(path+'/img'+str(index)+".jpg", "JPEG")

In [21]:
def make_directory(directory:str):
    parent_dir = "dataset/"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    return parent_dir+directory

In [88]:
for class_ in classes:
    image_links=get_links(class_)[:50]
    d,b64=separate_links(image_links)
    img_path=make_directory(class_)
    save_base64(b64,image_links,img_path)
    download_img(d,image_links,img_path)
    print('downloaded images for '+class_)

downloaded images for Eyeglasses
downloaded images for Frock
downloaded images for Glasses
downloaded images for Gloves
downloaded images for Handkerchief
downloaded images for Hat
downloaded images for Jacket
downloaded images for Jeans
downloaded images for Jinnah Cap
downloaded images for Kameez
downloaded images for Khussa
downloaded images for Kurta
downloaded images for Necktie
downloaded images for diaper
downloaded images for Pant
downloaded images for Purse
downloaded images for Raincoat
downloaded images for Sandals
downloaded images for Sari
downloaded images for School Uniform
downloaded images for Shalwar-Kameez
downloaded images for Sherwani
downloaded images for Shirt
downloaded images for Shoe
downloaded images for Sindhi Cap
downloaded images for Slippers
downloaded images for Socks
downloaded images for Sweater
downloaded images for T-Shirt
downloaded images for Waistcoat
downloaded images for Wallet
downloaded images for Watch
downloaded images for Wrap
downloaded im

## Cropping data to (224,224)

In [89]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from PIL import Image

# Define the target size
target_size = (224, 224)

# Define the source and destination folders
src_folder = "dataset"
dst_folder = "cropped"

# Loop through all files in the source folder
for filename in os.listdir(src_folder):
    for image_name in os.listdir(os.path.join(src_folder, filename)):
        try:
            # Load the image file using Keras' load_img function
            img = load_img(os.path.join(src_folder, filename, image_name), target_size=target_size)

            # Convert the image to a numpy array
            img = img_to_array(img)

            # Convert the numpy array back to a PIL Image object
            img = Image.fromarray(img.astype('uint8'))

            # Save the image to the destination folder
            os.makedirs(os.path.join(dst_folder, filename), exist_ok=True)
            img.save(os.path.join(dst_folder, filename, image_name))
        except Exception as e:
            print(f"Error processing image {image_name} in folder {filename}: {e}")


## Data Normalization

## SPLITTING INTO TRAIN AND TEST DATA (60-40)

In [3]:

# Set the root directory of your dataset
data_dir = 'dataset'

# Set the directory where you want to store the split dataset
split_dir = 'splitted'

# Create directories for the split dataset
os.makedirs(os.path.join(split_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(split_dir, 'val'), exist_ok=True)

# Set the percentage split
split_ratio = 0.6

# Loop through each class in the dataset
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)

    # Create a directory for the class in the split dataset
    os.makedirs(os.path.join(split_dir, 'train', class_name), exist_ok=True)
    os.makedirs(os.path.join(split_dir, 'val', class_name), exist_ok=True)

    # Get a list of image file names for the class
    image_list = os.listdir(class_dir)

    # Shuffle the list of image file names
    random.shuffle(image_list)

    # Split the image list into training and validation sets
    split_index = int(len(image_list) * split_ratio)
    train_images = image_list[:split_index]
    val_images = image_list[split_index:]

    # Copy the training images to the split dataset
    for image_name in train_images:
        src_path = os.path.join(class_dir, image_name)
        dst_path = os.path.join(split_dir, 'train', class_name, image_name)
        shutil.copy(src_path, dst_path)

    # Copy the validation images to the split dataset
    for image_name in val_images:
        src_path = os.path.join(class_dir, image_name)
        dst_path = os.path.join(split_dir, 'val', class_name, image_name)
        shutil.copy(src_path, dst_path)

NameError: name 'random' is not defined